Auteur: Michaël Leroy


Modification du traitement pour la création des edges graph communes, essai sur geodataframe pour profiter de la vectorisation:
        testé jusqu'a la recherche de paths... qui n'est pas vect et du coup hyper long (j'ai killé au bout de 3h)

    Création geodataframe Edges:

        calcul des edges attributes:
            - shortest path depuis le graph routier => distance, durée
            - geométrie: somme des LineString
            - length : somme des longeurs de chaque tronçons en mètres

[+] A faire:
        
        - prolonger les geometries des linestrings routes jusqu'au points centraux communes si on veut se servir des tracés sur une carte, sinon pas grave.

[-] Cons:
        
        40 min pour 250 nodes et 1219 edges

        -> temps estimé pour 170188 edges ~76h


In [1]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
# AREA

C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

Nodes = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.pkl'))
Nodes.index = Nodes.index.astype(int)

# Edges = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes0.pkl'))
Edges = gpd.read_feather(
    os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_C21.feather')
    )

In [3]:
Nodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 34466 entries, 2531584 to 2741108
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   y         34466 non-null  float64 
 1   x         34466 non-null  float64 
 2   codgeo    34466 non-null  object  
 3   area      34466 non-null  object  
 4   sleeping  15412 non-null  float64 
 5   visit     17923 non-null  float64 
 6   fooding   6616 non-null   float64 
 7   working   6950 non-null   float64 
 8   vehicule  4166 non-null   float64 
 9   geometry  34466 non-null  geometry
dtypes: float64(7), geometry(1), object(2)
memory usage: 2.9+ MB


In [4]:
Edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 205450 entries, (2531584, 2176035, 0) to (2741108, 2740963, 0)
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  205450 non-null  geometry
 1   src_trgt  205450 non-null  object  
 2   path      205450 non-null  object  
dtypes: geometry(1), object(2)
memory usage: 9.0+ MB


In [5]:
Nodes_routier = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl'))
Graph_routier = ox.graph_from_gdfs(
    Nodes_routier, 
    joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'))
    )

In [6]:
for u, v,  d in Graph_routier.edges(data=True):
    print(u, v, d)

    break

27174280 27174283 {'osmid': 8410390, 'lanes': '2', 'ref': 'D 592', 'name': 'Route de Belley', 'highway': 'secondary', 'oneway': False, 'reversed': False, 'length': 634.682, 'speed_kph': 62.2, 'travel_time': 36.7, 'geometry': <LINESTRING (5.61 45.612, 5.61 45.613, 5.61 45.615, 5.611 45.618, 5.611 45.618)>}


In [7]:
Edges = Edges.reset_index()
Edges.sample(2)

,u,v,key,geometry,src_trgt,path
160753,1110220,1109859,0,"LINESTRING (-0.39995 48.67643, -0.35369 48.64299)","[520186607, 520186235]","[520186607, 661027604, 520186235]"
28325,150417,153483,0,"LINESTRING (3.21528 49.58028, 3.25111 49.57222)","[457308792, 465192995]","[457308792, 445005772, 443803069, 3851940184, ..."


In [8]:
from tqdm.notebook import tqdm
tqdm.pandas()


# def get_path_length( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
def get_path_length( path):
    length = sum(
        ox.utils_graph.get_route_edge_attributes(
                    Graph_routier, 
                    path, 
                    attribute='length', 
                    minimize_key='length', 
                    retrieve_default=None)
    )
    return length



Edges['length'] = Edges.path.progress_apply(get_path_length)
Edges.sample(2)

  0%|          | 0/205450 [00:00<?, ?it/s]

,u,v,key,geometry,src_trgt,path,length
193875,1776907,1782133,0,"LINESTRING (1.93895 43.11282, 1.94711 43.08739)","[2428164830, 1621626180]","[2428164830, 392093489, 392093486, 411506524, ...",13582.795
27945,1828574,1446284,0,"LINESTRING (3.64890 45.21899, 3.71234 45.20049)","[1207766095, 1398754418]","[1207766095, 1207766205, 1207765979, 13799651,...",7791.388


In [9]:
from tqdm.notebook import tqdm
tqdm.pandas()


# def get_path_length( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
def get_path_length( path):
    length = sum(
        ox.utils_graph.get_route_edge_attributes(
                    Graph_routier, 
                    path, 
                    attribute='travel_time', 
                    minimize_key='length', 
                    retrieve_default=None)
    )
    return length



Edges['travel_time'] = Edges.path.progress_apply(get_path_length)
Edges.sample(2)

  0%|          | 0/205450 [00:00<?, ?it/s]

,u,v,key,geometry,src_trgt,path,length,travel_time
38879,131814,131783,0,"LINESTRING (4.61252 47.09742, 4.58960 47.08063)","[1988585738, 1430831514]","[1988585738, 1417287499, 1552686610, 155268651...",10660.23,543.5
166602,117249,1824928,0,"LINESTRING (1.20331 45.51275, 1.21414 45.61633)","[343389387, 1110583698]","[343389387, 1530967416, 343389367, 343389349, ...",12992.68,693.4


In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()
from shapely.geometry import LineString, MultiLineString
from shapely import ops


# def get_path_line( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
def get_path_line( path ):
    lines = ox.utils_graph.get_route_edge_attributes(
                    Graph_routier, 
                    path, 
                    attribute='geometry', 
                    minimize_key='length', 
                    retrieve_default=None)
    multi_line = MultiLineString(lines)
    merged_line = ops.linemerge(multi_line)
    return merged_line



Edges['path_line'] = Edges.path.progress_apply(get_path_line)
Edges.sample(2)

  0%|          | 0/205450 [00:00<?, ?it/s]

,u,v,key,geometry,src_trgt,path,length,travel_time,path_line
26787,2951027,2950973,0,"LINESTRING (3.53600 48.23073, 3.60898 48.22619)","[1526562487, 281440461]","[1526562487, 331201047, 1528585859, 1506607290...",5379.813,287.0,"LINESTRING (3.5363586 48.2355102, 3.5366067 48..."
68820,1223737,1223679,0,"LINESTRING (6.63623 47.21760, 6.64938 47.19069)","[2981919425, 458418727]","[2981919425, 410941920, 410941921, 410941928, ...",9800.820,563.9,"LINESTRING (6.6373781 47.2185384, 6.6374 47.21..."


# Verification

In [11]:
Edges.sample(2)

,u,v,key,geometry,src_trgt,path,length,travel_time,path_line
160135,2278193,445308,0,"LINESTRING (4.67809 45.41798, 4.67066 45.46490)","[2821525476, 8304791971]","[2821525476, 317623188, 1673495780, 338427311,...",9611.222,516.9,"LINESTRING (4.6781068 45.4186222, 4.6781989 45..."
2946,139672,139676,0,"LINESTRING (2.50143 51.04465, 2.52451 51.07100)","[10592613693, 2886871421]","[10592613693, 10592613617, 10592613601, 121591...",2755.846,181.9,"LINESTRING (2.5157612 51.0504233, 2.5157262 51..."


In [12]:
Edges.loc[20745]

u                                                         104971
v                                                         105042
key                                                            0
geometry       LINESTRING (6.2495954 46.2025309, 6.2919875 46...
src_trgt                                  [476282746, 307940770]
path           [476282746, 60272202, 2656068844, 2014579076, ...
length                                                  5560.047
travel_time                                                377.6
path_line      LINESTRING (6.2499822 46.2028589, 6.2500873 46...
Name: 20745, dtype: object

In [13]:
Nodes.loc[2930746]

y                                                   45.563927
x                                                    2.257384
codgeo                                                  19053
area        POLYGON ((2.2263709463164005 45.54121053994083...
sleeping                                                  NaN
visit                                                     NaN
fooding                                                   NaN
working                                                   NaN
vehicule                                                  NaN
geometry                 POINT (2.2573838620079094 45.563927)
Name: 2930746, dtype: object

In [14]:
Nodes.loc[2930944]

y                                                   45.605812
x                                                    2.276679
codgeo                                                  19233
area        POLYGON ((2.281758270702741 45.57340965460381,...
sleeping                                                  NaN
visit                                                     NaN
fooding                                                   NaN
working                                                   NaN
vehicule                                                  NaN
geometry                         POINT (2.2766793 45.6058115)
Name: 2930944, dtype: object

In [15]:
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup
from shapely.geometry import Point

map = folium.Map(location= [48.8, 2.3],
                    width='100%', height='100%', left='00%', top='0%', position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=0, max_zoom=18, zoom_start=10, 
                    min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, 
                    crs='EPSG3857', control_scale=True, prefer_canvas=False, 
                    no_touch=False, disable_3d=True, png_enabled=False, zoom_control=True)

# Start commune
folium.GeoJson(
    Nodes.loc[2930746,'area'], 
    tooltip = 'Source commune'  
                        # style_function=style_fn,
                        # popup = row.lib_rte,
                        # tooltip = row.lib_rte,

                ).add_to(map)

# Target commune
folium.GeoJson(
    Nodes.loc[2930944,'area'], 
    tooltip = 'Target commune'  
                        # style_function=style_fn,
                        # popup = row.lib_rte,
                        # tooltip = row.lib_rte,

                ).add_to(map)     

# Beefly edges
folium.GeoJson(
    Edges.loc[20745,'geometry'], 
    tooltip = 'Straight_line'
                        # style_function=style_fn,
                        # popup = row.lib_rte,
                        # tooltip = row.lib_rte,

                ).add_to(map)

# Road edges
folium.GeoJson(
    Edges.loc[20745,'path_line'], 
    tooltip = f"road_line: {Edges.loc[20745,'length']=} | {Edges.loc[20745,'travel_time']=}"
                        # style_function=style_fn,
                        # popup = row.lib_rte,
                        # tooltip = row.lib_rte,

                ).add_to(map)                

# # Graph routier src
# src_node = Graph_routier.nodes[1545567414]
# folium.Marker(
#     [src_node['y'],src_node['x']], 
#     tooltip = 'Graph routier source'   
#                         # style_function=style_fn,
#                         # popup = row.lib_rte,
#                         # tooltip = row.lib_rte,

#                 ).add_to(map) 

# # Graph routier trgt
# src_node = Graph_routier.nodes[1574182235]
# folium.Marker(
#     [src_node['y'],src_node['x']], 
#     tooltip = 'Graph routier target'    
#                         # style_function=style_fn,
#                         # popup = row.lib_rte,
#                         # tooltip = row.lib_rte,

#                 ).add_to(map) 

# Routes nodes 
for n in Edges.loc[20745,'path']:
    src_node = Graph_routier.nodes[n]
    folium.Marker(
        [src_node['y'],src_node['x']], 
        tooltip = n    
                            # style_function=style_fn,
                            # popup = row.lib_rte,
                            # tooltip = row.lib_rte,

                    ).add_to(map) 

display(map)
del map                


In [16]:
joblib.dump(
    Edges.set_index(['u','v','key']),
    os.path.join(
                CFG.data_path, 
                f'[{CFG.area_name}]Gedges_communes_C22.pkl'
            )
)

['./osm_datas/[france]Gedges_communes_C22.pkl']

--------------------------------